# Notes
* Pure python implementation using Polars LazyFrame

# Libraries

In [ ]:
import polars as pl
import sempy.fabric as fabric
import notebookutils as nbutl

from datetime import datetime
import pytz

from typing import *
from dataclasses import dataclass

# Parameters

In [ ]:
# list of workspaces to be scanned, support both workspaceId(recommended) and workspaceName
workspaceList = ['workspace1', 'workspace2']

# Define Destination Delta Table path that stores Fabric Schedules Metadata
destTablePath = 'abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/{schemaName}/{tableName}' # Replace with actual delta table path

# Datetime Parameters
timezone_LCL = 'Australia/Adelaide' # Replace with your local timezone, https://en.wikipedia.org/wiki/List_of_tz_database_time_zones
zUPD_UTC, zUPD_LCL = datetime.now(), datetime.now(pytz.timezone(timezone_LCL))

# Configuration

In [ ]:
# Define ItemKey dataclass structure
@dataclass
class ItemKey:
    DefaultKey: str
    ScheduleKey: str
    UrlKey: str

In [ ]:
# Define items keys for REST API calls for supported item types
ITEM_KEYS = [
    ItemKey('DataPipeline', 'Pipeline', 'pipelines'),
    ItemKey('CopyJob', 'CopyJob', 'copyjobs'),
    ItemKey('Notebook', 'RunNotebook', 'synapsenotebooks'),
    ItemKey('SparkJobDefinition', 'sparkjob', 'sparkjobdefinitions'),
    ItemKey('Dataflow', 'Refresh', 'dataflows-gen2')   
]

# Convert ItemKeys to dictionary for quick lookup
ITEM_KEY_LOOKUP = {item.DefaultKey: item for item in ITEM_KEYS}

# Functions

In [ ]:
# -------------------- Utilities -------------------- #
def convert_ordinal(n: int) -> str:
    """Convert integer to its ordinal representation."""
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f'{n}{suffix}'


# -------------------- API Wrappers -------------------- #
def get_workspace_items(client, workspace_id, item_key_lookup):
    """Fetch valid items from a workspace based on known types."""
    items = client.get(f'/v1/workspaces/{workspace_id}/items').json().get('value', [])
    valid_types = set(item_key_lookup.keys())
    return [i for i in items if i['type'] in valid_types]


def get_item_schedules(client, workspace_id, item_id, schedule_key):
    """Fetch schedules for a given item."""
    try:
        return client.get(f"/v1/workspaces/{workspace_id}/items/{item_id}/jobs/{schedule_key}/schedules").json().get('value', [])
    except Exception:
        return []


def get_last_run(client, workspace_id, item_id):
    """Fetch the most recent run for an item."""
    runs = client.get(f"/v1/workspaces/{workspace_id}/items/{item_id}/jobs/instances").json().get('value', [])
    return max(runs, key=lambda r: r['startTimeUtc'], default={'startTimeUtc': None, 'endTimeUtc': None, 'status': 'N.A.'})


# -------------------- Metadata Parser -------------------- #
def parse_schedule_metadata(item, schedule, last_run, workspace_id, workspace_name, item_key):
    """Parse and format schedule metadata for reporting."""
    config = schedule['configuration']
    schedule_type = config['type']

    if schedule_type == 'Cron':
        schedule_type = 'By Minutes'
    elif schedule_type == 'Weekly':
        schedule_type += ': ' + ', '.join(config['weekdays'])
    elif schedule_type == 'Monthly':
        schedule_type += f": Every {config['recurrence']} month{'s' if config['recurrence'] > 1 else ''}"
        occ = config['occurrence']
        if occ['occurrenceType'] == 'DayOfMonth':
            schedule_type += f" on the {convert_ordinal(occ['dayOfMonth'])} day"
        elif occ['occurrenceType'] == 'OrdinalWeekday':
            schedule_type += f" on the {occ['weekIndex']} {occ['weekday']}"

    scheduled_times = (
        [f"Every {config['interval']} Minutes"]
        if config['type'] == 'Cron'
        else config['times']
    )

    return {
        'ItemId': item['id'],
        'ItemName': item['displayName'],
        'ItemType': item['type'],
        'ItemDescription': item['description'],
        'ScheduleId': schedule['id'],
        'ScheduleEnabled': schedule['enabled'],
        'ScheduleType': schedule_type,
        'ScheduledTimes': scheduled_times,
        'ScheduleTimezone': config['localTimeZoneId'],
        'LastRun_StartTime_UTC': last_run['startTimeUtc'],
        'LastRun_EndTime_UTC': last_run['endTimeUtc'],
        'LastRun_Status': last_run['status'],
        'ScheduleCreatedTime_UTC': schedule['createdDateTime'],
        'ScheduleStartTime_LCL': config['startDateTime'],
        'ScheduleEndTime_LCL': config['endDateTime'],
        'WorkspaceId': workspace_id,
        'WorkspaceName': workspace_name,
        'ItemURL': f"https://app.powerbi.com/groups/{workspace_id}/{item_key.UrlKey}/{item['id']}?experience=power-bi"
    }


# -------------------- Main Scanner -------------------- #
def scan_item_schedule(workspace, item_key_lookup=ITEM_KEY_LOOKUP, update_time=zUPD_LCL, local_tz=timezone_LCL):
    """Scan schedules for all valid items in a workspace and return a Polars DataFrame."""
    client = fabric.FabricRestClient()
    workspace_name, workspace_id = fabric.resolve_workspace_name_and_id(workspace)
    items = get_workspace_items(client, workspace_id, item_key_lookup)

    records = []
    for item in items:
        item_key = item_key_lookup[item['type']]
        schedules = get_item_schedules(client, workspace_id, item['id'], item_key.ScheduleKey)
        last_run = get_last_run(client, workspace_id, item['id'])

        for schedule in schedules:
            if schedule:
                metadata = parse_schedule_metadata(item, schedule, last_run, workspace_id, workspace_name, item_key)
                records.append(metadata)

    df_lazy = pl.LazyFrame(records)

    datetime_cols = [
        'LastRun_StartTime_UTC', 'LastRun_EndTime_UTC',
        'ScheduleCreatedTime_UTC', 'ScheduleStartTime_LCL', 'ScheduleEndTime_LCL'
    ]

    df_lazy = (
        df_lazy.filter(pl.col('ScheduleEnabled') == True) # remove this filter if you wants to see disabled schedules as well
          .with_columns([
              pl.col(datetime_cols).cast(pl.Datetime, strict=False).dt.replace_time_zone('UTC'),
              pl.when(pl.col('ScheduleType') == 'Cron')
                .then(pl.col('ScheduledTimes').list.first())
                .otherwise(pl.col('ScheduledTimes').list.join(', '))
                .cast(pl.String, strict=False)
                .alias('ScheduledTimes'),
              pl.lit(update_time).dt.replace_time_zone('UTC').alias('zUPD')
          ])
          .with_columns([
              pl.col('LastRun_StartTime_UTC').dt.convert_time_zone(local_tz).dt.replace_time_zone('UTC').alias('LastRun_StartTime_LCL'), # workaround to avoid TIMEZONE_NTZ error
              pl.col('LastRun_EndTime_UTC').dt.convert_time_zone(local_tz).dt.replace_time_zone('UTC').alias('LastRun_EndTime_LCL'), # workaround to avoid TIMEZONE_NTZ error
              pl.col('ScheduleCreatedTime_UTC').dt.convert_time_zone(local_tz).dt.replace_time_zone('UTC').alias('ScheduleCreatedTime_LCL') # workaround to avoid TIMEZONE_NTZ error
          ])
          .select([
            'ItemId','ItemName','ItemType','ItemDescription',
            'WorkspaceId','WorkspaceName',
            'ScheduleEnabled','ScheduleId','ScheduleType',
            'ScheduledTimes','ScheduleTimezone',
            'LastRun_StartTime_LCL','LastRun_EndTime_LCL','LastRun_Status',
            'ScheduleStartTime_LCL','ScheduleEndTime_LCL',
            'ScheduleCreatedTime_LCL',
            'ItemURL','zUPD'
        ])
        .sort(pl.col('LastRun_StartTime_LCL'), descending = True)
    )

    return df_lazy

# Main

In [ ]:
# concat workspace item schedules into one dataframe
df_schedules = pl.concat([scan_item_schedule(w) for w in workspaceList], how = 'vertical').collect()

# Evaluation

In [ ]:
# display(df_schedules)

# Export

In [ ]:
# Write Fabric Schedules to a Delta Table
df_schedules.write_delta(
    destTablePath,
    mode = 'overwrite',
#    delta_write_options={'schema_mode': 'overwrite'}
    )